In [37]:
# 🚀 Install required packages
#!pip install -q sentence-transformers imbalanced-learn

In [38]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import KFold
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
import joblib
from sentence_transformers import SentenceTransformer

In [39]:
# NLTK setup
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [40]:
# Global config
emotion_cols = ['anger', 'fear', 'joy', 'sadness', 'surprise']
stop_words = set(stopwords.words('english')) - {'not', 'no', 'never'}
lemmatizer = WordNetLemmatizer()


In [41]:
# Preprocessing function
def preprocess_text(text):
    text = str(text).lower()
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)
    text = re.sub(r'[^a-z\s]', '', text)
    tokens = text.split()
    tokens = [lemmatizer.lemmatize(w) for w in tokens if w not in stop_words]
    return ' '.join(tokens)


In [42]:
# Load data
df = pd.read_csv('/track-a.csv')  # Adjust path if needed
df['clean_text'] = df['text'].apply(preprocess_text)


In [43]:
# Embedding
embedder = SentenceTransformer('all-MiniLM-L6-v2')
X = embedder.encode(df['text'].tolist(), show_progress_bar=True)
y = df[emotion_cols].values
joblib.dump(embedder, 'embedder.joblib')


Batches:   0%|          | 0/87 [00:00<?, ?it/s]

['embedder.joblib']

In [44]:
# KFold and Threshold tuning
kf = KFold(n_splits=5, shuffle=True, random_state=42)
best_thresholds = {emo: [] for emo in emotion_cols}
metrics_summary = {emo: {'precision': [], 'recall': [], 'f1': []} for emo in emotion_cols}

def find_best_thresholds(X_val, y_val, model):
    thresh = {}
    probs = model.predict_proba(X_val)  # list of arrays, one per emotion
    for i, emo in enumerate(emotion_cols):
        prob_pos = probs[i][:, 1]
        best, best_f1 = 0.5, 0
        for t in np.arange(0.3, 0.91, 0.05):
            f = f1_score(y_val[:, i], prob_pos >= t, zero_division=0)
            if f > best_f1:
                best, best_f1 = t, f
        thresh[emo] = best
    return thresh

In [45]:
# Train with folds
for fold, (train_i, val_i) in enumerate(kf.split(X)):
    X_tr, X_val = X[train_i], X[val_i]
    y_tr, y_val = y[train_i], y[val_i]

    base_clf = BalancedRandomForestClassifier(random_state=42, n_estimators=100)
    multi_clf = MultiOutputClassifier(base_clf)
    multi_clf.fit(X_tr, y_tr)

    fold_thresh = find_best_thresholds(X_val, y_val, multi_clf)
    for emo in emotion_cols:
        best_thresholds[emo].append(fold_thresh[emo])

    # Evaluate metrics on validation set
    probs = multi_clf.predict_proba(X_val)
    for i, emo in enumerate(emotion_cols):
        pred_bin = (probs[i][:, 1] >= fold_thresh[emo]).astype(int)
        metrics_summary[emo]['precision'].append(precision_score(y_val[:, i], pred_bin, zero_division=0))
        metrics_summary[emo]['recall'].append(recall_score(y_val[:, i], pred_bin, zero_division=0))
        metrics_summary[emo]['f1'].append(f1_score(y_val[:, i], pred_bin, zero_division=0))


In [46]:
# Report final thresholds and metrics
final_thresholds = {emo: np.mean(best_thresholds[emo]) for emo in emotion_cols}
print("\n📊 Tuned thresholds:")
for emo, val in final_thresholds.items():
    print(f"{emo}: {val:.3f}")
joblib.dump(final_thresholds, 'thresholds.joblib')



📊 Tuned thresholds:
anger: 0.470
fear: 0.440
joy: 0.450
sadness: 0.440
surprise: 0.460


['thresholds.joblib']

In [47]:
print("\n📈 Cross-validated performance:")
for emo in emotion_cols:
    p = np.mean(metrics_summary[emo]['precision'])
    r = np.mean(metrics_summary[emo]['recall'])
    f = np.mean(metrics_summary[emo]['f1'])
    print(f"{emo}: Precision={p:.3f}, Recall={r:.3f}, F1={f:.3f}")



📈 Cross-validated performance:
anger: Precision=0.361, Recall=0.523, F1=0.418
fear: Precision=0.673, Recall=0.915, F1=0.775
joy: Precision=0.509, Recall=0.692, F1=0.582
sadness: Precision=0.547, Recall=0.710, F1=0.615
surprise: Precision=0.546, Recall=0.698, F1=0.610


In [48]:
# Train final model
final_model = MultiOutputClassifier(BalancedRandomForestClassifier(random_state=42, n_estimators=100))
final_model.fit(X, y)
joblib.dump(final_model, 'multi_rf_model.joblib')
print("\n✅ Final multi-label model saved!")


✅ Final multi-label model saved!


In [49]:
def predict_emotions(sent_list):
    embedder = joblib.load('embedder.joblib')
    thresholds = joblib.load('thresholds.joblib')
    model = joblib.load('multi_rf_model.joblib')
    X_new = embedder.encode(sent_list)
    probs = model.predict_proba(X_new)
    preds = []
    for i in range(len(sent_list)):
        pred = {}
        for j, emo in enumerate(emotion_cols):
            prob_pos = probs[j][i, 1]
            pred[emo] = int(prob_pos >= thresholds[emo])
        preds.append(pred)
    return preds

In [50]:
# Train final model
final_model = MultiOutputClassifier(BalancedRandomForestClassifier(random_state=42, n_estimators=100))
final_model.fit(X, y)
joblib.dump(final_model, 'multi_rf_model.joblib')
print("\n✅ Final multi-label model saved!")


✅ Final multi-label model saved!


In [51]:
# Interactive demo
if __name__ == "__main__":
    input_text = input("Enter sentences separated by '|': ")
    sents = [s.strip() for s in input_text.split('|')]
    results = predict_emotions(sents)
    for sent, pred in zip(sents, results):
        print(f"\nSentence: {sent}")
        for emo in emotion_cols:
            print(f"  {emo}: {pred[emo]}")

Enter sentences separated by '|': I couldn’t believe I actually won the competition — I was thrilled and completely surprised, but also nervous about what comes next.

Sentence: I couldn’t believe I actually won the competition — I was thrilled and completely surprised, but also nervous about what comes next.
  anger: 0
  fear: 0
  joy: 1
  sadness: 0
  surprise: 0
